In [ ]:
import pandas as pd
import re

# Function to load and clean one file
def load_and_clean_ecec_file(file_path, year):
    df_raw = pd.read_excel(file_path, header=[0, 1], skiprows=5)

    # Split into metadata and timestamps
    meta_df = df_raw.iloc[:, :7].copy()
    timestamps_df = df_raw.iloc[:, 7:].copy()

    # Flatten column names
    meta_df.columns = ['_'.join([str(c) for c in col if 'Unnamed' not in str(c)]).strip() for col in meta_df.columns]
    timestamps_df.columns = [f"{str(c[0]).strip()}_{str(c[1]).strip()}" for c in timestamps_df.columns]

    # Add row index for merge
    meta_df['row_id'] = meta_df.index
    timestamps_df['row_id'] = timestamps_df.index

    # Melt timestamp columns
    long_df = timestamps_df.melt(id_vars='row_id', var_name='date_inout', value_name='timestamp')
    long_df[['date_str', 'in_out']] = long_df['date_inout'].str.extract(r'(.*)_(IN|OUT)', expand=True)

    # Add year and convert to datetime
    long_df['date_str'] = long_df['date_str'].str.strip() + f' {year}'
    long_df['date'] = pd.to_datetime(long_df['date_str'], format='%b %d %Y', errors='coerce')

    # Pivot to get IN and OUT columns
    pivot_df = long_df.pivot_table(index=['row_id', 'date'], columns='in_out', values='timestamp', aggfunc='first').reset_index()

    for col in ['IN', 'OUT']:
        if col not in pivot_df.columns:
            pivot_df[col] = pd.NaT

    # Merge with metadata
    final_df = pivot_df.merge(meta_df, on='row_id', how='left')

    # Clean time strings
    def clean_time_only(x):
        if pd.isna(x):
            return x
        match = re.match(r'^\s*\d{1,2}:\d{2}\s*(AM|PM)', str(x), re.IGNORECASE)
        return match.group(0) if match else x

    final_df['IN'] = final_df['IN'].apply(clean_time_only)
    final_df['OUT'] = final_df['OUT'].apply(clean_time_only)

    # Add year column for tracking
    final_df['year'] = int(year)

    return final_df[['Record ID', 'Student Status', 'Room', 'Tags', 'date', 'IN', 'OUT', 'year']]

# File URLs and corresponding years
ecec_files = {
    "2022": "https://raw.githubusercontent.com/duehl85/econ8310_semester_project/main/CSIData/ECEC%202022%20Student%20Sign%20In%20and%20Out.xlsx",
    "2023": "https://raw.githubusercontent.com/duehl85/econ8310_semester_project/main/CSIData/ECEC%202023%20Student%20Sign%20In%20and%20Out.xlsx",
    "2024": "https://raw.githubusercontent.com/duehl85/econ8310_semester_project/main/CSIData/ECEC%202024%20Student%20Sign%20In%20and%20Out.xlsx",
    "2025": "https://raw.githubusercontent.com/duehl85/econ8310_semester_project/main/CSIData/ECEC%202025%2001012025-02282025%20Student%20Sign%20In%20and%20Out.xlsx"
}

# Process all files and combine
all_years_df = pd.concat(
    [load_and_clean_ecec_file(path, year) for year, path in ecec_files.items()],
    ignore_index=True
)

#regular expression pattern to identify age groups

age_group_pattern = r'(Infants|Multi-Age|Toddlers|Preschool|Pre-K)'

#extract age group from room name and create new column called age_group

all_years_df['age_group'] = all_years_df['Room'].str.extract(age_group_pattern, expand=False)

#41 out of 7,515 out entries, there are 41 entries where the timestamp is -- (possibly a student was not clocked out)
#0.55% is not a material amount of entries. Treating these out times as NAN

all_years_df['OUT'] = all_years_df['OUT'].replace('--', pd.NA)


In [80]:
from datetime import datetime, timedelta

# Make sure IN and OUT are datetime objects
all_years_df['in_datetime'] = pd.to_datetime(all_years_df['date'].astype(str) + ' ' + all_years_df['IN'], errors='coerce')
all_years_df['out_datetime'] = pd.to_datetime(all_years_df['date'].astype(str) + ' ' + all_years_df['OUT'], errors='coerce')

# Drop rows where IN or OUT is missing (or handle imputation later)
attended_df = all_years_df.dropna(subset=['in_datetime', 'out_datetime']).copy()

# Function to generate 30-min blocks between IN and OUT
def generate_30min_blocks(start, end):
    return pd.date_range(start=start, end=end, freq='30min').tolist()

# Apply to each row
attended_df['time_blocks'] = attended_df.apply(lambda row: generate_30min_blocks(row['in_datetime'], row['out_datetime']), axis=1)

# Explode so each row = one 30-min block per student
expanded_df = attended_df.explode('time_blocks')

# Round the block timestamp to the nearest floor 30-min for consistency
expanded_df['time_block'] = expanded_df['time_blocks'].dt.floor('30min')

# Final attendance time grid: One row = one child in one room at one 30-min block
attendance_grid = expanded_df[['Record ID', 'Student Status', 'age_group', 'time_block']].copy()

/var/folders/bd/pql52mvd73zd_wtzsgm40l4h0000gn/T/ipykernel_48960/403524351.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_years_df['out_datetime'] = pd.to_datetime(all_years_df['date'].astype(str) + ' ' + all_years_df['OUT'], errors='coerce')


In [100]:
import numpy as np

#group students by age group and time block

grouped = attendance_grid.groupby(['age_group', 'time_block', 'Student Status']).agg(
    children_present=('Record ID', 'nunique')
).reset_index()

#define the student/staff ratios

ratio_table = pd.DataFrame({
    'age_group': ['Infants', 'Multi-Age', 'Toddlers', 'Preschool', 'Pre-K'],
    'student_to_staff': [4, 4, 6, 10, 12]
})

#merge the two dfs based on age_group

grouped = grouped.merge(ratio_table, on='age_group', how='left')

#determine staffing required. Use np ceiling function to round up to next integer

grouped['staff_required'] = np.ceil(grouped['children_present'] / grouped['student_to_staff']).astype(int)



In [101]:
grouped.to_csv("ecec_staffing_grouped.csv", index=False)